# **LangChain Search**

Install the necessary packages

In [2]:
!python --version

Python 3.10.12


In [3]:
!pip install streamlit
!pip install langchain
!pip install google-cloud-aiplatform
!pip install chromadb
!pip install transformers
!pip install watchdog


[notice] A new release of pip is available: 23.0.1 -> 23.1.2
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.0.1 -> 23.1.2
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.0.1 -> 23.1.2
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.0.1 -> 23.1.2
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.0.1 -> 23.1.2
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.0.1 -> 23.1.2
[notice] To update, run: pip install --upgrade pip


Create a Streamlit app and save to a local file

In [4]:
!gcloud auth login

Your browser has been opened to visit:

    https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=32555940559.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A8085%2F&scope=openid+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fuserinfo.email+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcloud-platform+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fappengine.admin+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fsqlservice.login+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcompute+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Faccounts.reauth&state=zFETlcDmFZUjeZbMMYae7GoUQsgINq&access_type=offline&code_challenge=ToGaJDHlu_tAyvVT21UtFfkB-4YSXmvr66zVUeHKY68&code_challenge_method=S256


You are now logged in as [nuttee@google.com].
Your current project is [cloud-llm-preview4].  You can change this setting by running:
  $ gcloud config set project PROJECT_ID


In [5]:
!gcloud config set project cloud-llm-preview4
!gcloud auth application-default login

Updated property [core/project].
Your browser has been opened to visit:

    https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=764086051850-6qr4p6gpi6hn506pt8ejuq83di341hur.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A8085%2F&scope=openid+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fuserinfo.email+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcloud-platform+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fsqlservice.login+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Faccounts.reauth&state=bTpG6neBljhEnDVTXb9Nmwgeqnkp2s&access_type=offline&code_challenge=zfK4WxiN1KgMue9AmEivoGXdF5Ryj02jOx-g_83ImVc&code_challenge_method=S256


Credentials saved to file: [/Users/nuttee/.config/gcloud/application_default_credentials.json]

These credentials will be used by any library that requests Application Default Credentials (ADC).

Quota project "cloud-llm-preview4" was added to ADC which can be used by Google client libraries for billing and quota. Note that some services may 

In [11]:
import streamlit as st
import vertexai
from langchain import PromptTemplate
from langchain import VectorDBQA
from langchain.document_loaders import TextLoader
from langchain.embeddings import VertexAIEmbeddings
from langchain.llms import VertexAI
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import Chroma


TypeError: issubclass() arg 1 must be a class

In [6]:
# Initialize Vertex AI
vertexai.init(project="cloud-llm-preview4", location="us-central1")

llm = VertexAI(
    model_name="text-bison@001",
    max_output_tokens=1024,
    temperature=0.1,
    top_p=0.8,
    top_k=40,
    verbose=True,)
    
embeddings = VertexAIEmbeddings()

In [17]:
loader = TextLoader("text_source/factsheet_xpresscash.txt")
documents = loader.load()
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
texts = text_splitter.split_documents(documents)

embeddings = VertexAIEmbeddings()
xpress_cash_store = Chroma.from_documents(
    texts, embeddings, collection_name="xpress-cash"
)

Created a chunk of size 1697, which is longer than the specified 1000
Created a chunk of size 1854, which is longer than the specified 1000
Created a chunk of size 1672, which is longer than the specified 1000
Created a chunk of size 2612, which is longer than the specified 1000
Created a chunk of size 1618, which is longer than the specified 1000
Created a chunk of size 1926, which is longer than the specified 1000
Created a chunk of size 1880, which is longer than the specified 1000
Created a chunk of size 1039, which is longer than the specified 1000
Created a chunk of size 1428, which is longer than the specified 1000


In [18]:
# Toolkit and Agent
from langchain.agents.agent_toolkits import (
    create_vectorstore_agent,
    VectorStoreToolkit,
    VectorStoreInfo,
)

vectorstore_info = VectorStoreInfo(
    name="xpress_cash",
    description="Xpress Cash finance product for quick loan",
    vectorstore=xpress_cash_store,
)
toolkit = VectorStoreToolkit(vectorstore_info=vectorstore_info, llm=llm)
agent_executor = create_vectorstore_agent(llm=llm, toolkit=toolkit, verbose=True)

In [19]:
question="""
Instruction: Give an answer in Thai language
ผลิตภัณฑ์: xpress_cash
จุดประสงค์: คุณสมบัติ/เงื่อนไขการสมัคร 

ไม่ได้ทำงานประจำ สมัครสินเชื่อเงินด่วนได้ไหม
"""

agent_executor.run(question)



> Entering new  chain...
I don't know what to do.
Action: 
Action Input: 
Observation:  is not a valid tool, try another one.
Thought:I don't know what to do.
Action: 
Action Input: 
Observation:  is not a valid tool, try another one.
Thought:I don't know what to do.
Action: 
Action Input: 
Observation:  is not a valid tool, try another one.
Thought:

KeyboardInterrupt: 

In [ ]:
# Streamlit app
st.title('LangChain Search')

# Check if the 'Search' button is clicked
if st.button("Search"):
    # Validate inputs
    if not openai_api_key.strip() or not serpapi_api_key.strip() or not search_query.strip():
        st.write(f"Please provide the missing fields.")
    else:
        try:
            # Initialize the OpenAI module, load the SerpApi tool, and run the search query using an agent
            llm=OpenAI(temperature=0, openai_api_key=openai_api_key, verbose=True)
            tools = load_tools(["serpapi"], llm, serpapi_api_key=serpapi_api_key)
            agent = initialize_agent(tools, llm, agent="zero-shot-react-description", verbose=True)

            result = agent.run(search_query)
            
            st.write(result)
        except Exception as e:
            st.write(f"An error occurred: {e}")

Install localtunnel to serve the Streamlit app

In [ ]:
!npm install localtunnel

Run the Streamlit app in the background

In [4]:
!streamlit run app.py &>/content/logs.txt &

Expose the Streamlit app on port 8501

In [ ]:
!npx localtunnel --port 8501